In [17]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
import os
import pandas as pd
pd.set_option('display.max_rows',None)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 as cv
from tqdm import tqdm
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [19]:
df = pd.read_csv('augmented_data.csv')

In [10]:
label_to_disease = ["Cassava Bacterial Blight (CBB)", "Cassava Brown Streak Disease (CBSD)", "Cassava Green Mottle (CGM)", "Cassava Mosaic Disease (CMD)", "Healthy"]

### RandomForest 

In [20]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

num_classes = 5


model = RandomForestClassifier(n_estimators=100, random_state=42)


# Load and preprocess images
X = []
y = []

target_size = (200, 150)  # Width, Height

for index, row in tqdm(df.iterrows()):
    img = cv.imread(row['image_id'])
    
    # resizing images
    resized_img = cv.resize(img, target_size)

    # min-max normalization
    normalized_image = resized_img / 255.0

    X.append(normalized_image)
    y.append(row['label'])


X = np.array(X)
y = np.array(y)


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

50768it [08:40, 97.54it/s] 


In [21]:
X_train.shape, y.shape

((40614, 150, 200, 3), (50768,))

In [22]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


X_train_reshaped = X_train.reshape(X_train.shape[0], -1)

X_train_flattened = X_train.reshape(X_train.shape[0], -1)

X_test_flattened = X_test.reshape(X_test.shape[0], -1)


rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train_flattened, y_train)

y_pred = rf_classifier.predict(X_test_flattened)

## Evaluation

In [23]:
# Calculate accuracy
accuracy = rf_classifier.score(X_test_flattened, y_test)
print("Test Accuracy:", accuracy)

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)


Test Accuracy: 0.5124089028954106
Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.55      0.51      2389
           1       0.42      0.27      0.33      2428
           2       0.47      0.44      0.46      2396
           3       0.60      0.90      0.72      2399
           4       0.33      0.00      0.00       542

    accuracy                           0.51     10154
   macro avg       0.46      0.43      0.41     10154
weighted avg       0.49      0.51      0.48     10154

Confusion Matrix:
[[1310  415  448  216    0]
 [ 783  660  616  368    1]
 [ 505  378 1064  449    0]
 [  70   74   86 2168    1]
 [  51   26   53  411    1]]
